In [1]:
include("src/sgs_store.jl")

using ..HllGrad
using ..HllSets

using PyCall
using CSV
using DataFrames
using WordTokenizers

redis   = pyimport("redis")
# Connect to Redis
r = redis.Redis(host="localhost", port=6379, db=0)

csv_file_path = "/home/alexmy/Downloads/POC/DATA/enron_05_17_2015_with_labels_v2.csv"

println(typeof(r))


PyObject


In [2]:
df      = DataFrame(CSV.File(csv_file_path, header=true))
df.From = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.From)
df.To   = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.To)

df_filled = coalesce.(df, "unknown")

print()

In [3]:
"""
    1. Message-ID	
    2. Date	
    3. From	
    4. To	
    5. Subject	
    6. X-From	X-To	X-cc	X-bcc	X-Folder	X-Origin	X-FileName	
    13. content	
    14. user	
    15. Cat_1_level_1	Cat_1_level_2	Cat_1_weight	Cat_2_level_1	Cat_2_level_2	Cat_2_weight	Cat_3_level_1	Cat_3_level_2	Cat_3_weight	Cat_4_level_1	Cat_4_level_2	Cat_4_weight	Cat_5_level_1	Cat_5_level_2	Cat_5_weight	Cat_6_level_1	Cat_6_level_2	Cat_6_weight	Cat_7_level_1	Cat_7_level_2	Cat_7_weight	Cat_8_level_1	Cat_8_level_2	Cat_8_weight	Cat_9_level_1	Cat_9_level_2	Cat_9_weight	Cat_10_level_1	Cat_10_level_2	Cat_10_weight	Cat_11_level_1	Cat_11_level_2	Cat_11_weight	Cat_12_level_1	Cat_12_level_2	Cat_12_weight	
    51. labeled
"""

# Initialize sets to collect data from specific columns
hll_03 = HllSets.HllSet{10}()     # Set{String}()
hll_04 = HllSets.HllSet{10}()     # Set{String}()
hll_05 = HllSets.HllSet{10}()     # Set{String}()
hll_06 = HllSets.HllSet{10}()     # Set{String}()
hll_14 = HllSets.HllSet{10}()     # Set{String}()
hll_15 = HllSets.HllSet{10}()     # Set{String}()


HllSet{10}()


In [5]:
using ..Store
using ..Util
using JSON3
using TextAnalysis
using WordTokenizers
using Base.Threads

tokenizer = WordTokenizers.Words
# r::PyObject, df::DataFrame, parent::String, cols::Vector; p::Int=10, chunk_size::Int=512000
# Store.ingest_df(r, tokenizer, df, csv_file_path, [:From, :To, :Subject, :content, :user])

cols = [:From, :To, :Subject, :content, :user]
p::Int=10 
chunk_size::Int=512000
parent = csv_file_path

for column in cols    
    col_values  = df[:, column]
    col_sha1    = Util.sha1_union([parent, string(column)])
    column_size = Base.summarysize(col_values)
    num_chunks  = ceil(Int, column_size / chunk_size)
    chunks      = Store.chunk_array(col_values, num_chunks)

    println(col_sha1, "; num_chunks: ", num_chunks)
    dataset = Store.ingest_df_column(r, tokenizer, chunks, col_sha1)
    println("Column dataset: ", dataset)
end


6f983ba3758e7233f7379a9c7b6ee565808a8de6; num_chunks: 79
Column dataset: [26, 6, 22, 6, 170, 6, 14, 18, 22, 14, 22, 46, 6, 22, 2, 2, 2, 1034, 18, 1102, 38, 2, 14, 14, 6, 38, 6, 22, 2, 62, 38, 542, 2, 174, 14, 10, 26, 142, 46, 6, 6, 38, 14, 12, 514, 530, 6, 6, 6, 22, 30, 130, 62, 6, 30, 54, 14, 10, 6, 22, 14, 14, 14, 118, 50, 30, 10, 20, 62, 22, 14, 10, 54, 22, 46, 6, 22, 6, 54, 14, 14, 6, 38, 22, 14, 14, 142, 70, 14, 14, 266, 14, 14, 38, 30, 262, 94, 6, 6, 14, 522, 6, 2, 0, 38, 62, 2, 102, 6, 6, 30, 18, 14, 14, 30, 6, 12, 6, 22, 70, 28, 14, 518, 6, 6, 134, 26, 26, 130, 54, 10, 2, 6, 14, 14, 18, 14, 30, 158, 70, 6, 30, 6, 22, 54, 20, 62, 158, 394, 14, 8, 26, 30, 6, 14, 10, 14, 6, 22, 30, 16, 1054, 46, 26, 14, 14, 6, 54, 266, 10, 526, 22, 62, 158, 34, 22, 34, 30, 10, 18, 30, 22, 14, 66, 70, 14, 22, 2, 62, 10, 26, 6, 10, 38, 6, 6, 30, 142, 18, 94, 22, 46, 134, 38, 30, 22, 42, 2342, 14, 30, 14, 6, 14, 18, 30, 14, 70, 74, 22, 518, 22, 34, 14, 22, 6, 2, 22, 86, 14, 14, 14, 6, 6, 4, 30, 14, 1

In [ ]:
using Random
using Distributions

# Example term frequencies (tf)
terms = ["term1", "term2", "term3", "term4", "term5"]
tf = [10, 20, 30, 25, 15]

# Calculate the total term frequency
total_tf = sum(tf)

# Calculate the probability distribution
probabilities = tf ./ total_tf

# Create a categorical distribution based on the probabilities
term_distribution = Categorical(probabilities)

# Function to sample terms based on the distribution
function sample_terms(terms, term_distribution, num_samples)
    sampled_terms = []
    for _ in 1:num_samples
        term_index = rand(term_distribution)
        push!(sampled_terms, terms[term_index])
    end
    return sampled_terms
end

# Sample 10 terms from the collection
num_samples = 3
sampled_terms = sample_terms(terms, term_distribution, num_samples)

println("Sampled terms: ", sampled_terms)